# Synthèse de textures par réseau convolutif (filtres aléatoires)

## Mise en Place

In [ ]:
import os
from urllib.request import urlopen
from io import BytesIO

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms.functional as TF

import utils

In [ ]:
# fetch images
texture_imgnames = ["bois.png", "briques.png", "mur.png",
                    "tissu.png", "nuages.png", "pebbles.jpg", "wall1003.png"]
#TODO use urllib instead 
#import wget
for fname in texture_imgnames:
    os.system("wget -c https://www.idpoisson.fr/galerne/mva/" + fname)

In [ ]:
# device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is", device)

## VGG

In [ ]:
#TODO description de VGG

Dans un premier temps, nous chargeons le modèle VGG pré-entraîné afin d'évaluer notre implémentation sur un réseau dont les poids sont déjà optimisés:

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

## Reconstruction d'images

He *et al.* définissent une manière de reconstruire une image $x_0$ à l'aide d'une couche $l$ d'un réseau de neurones et d'une pré-image (synthétique) $x$. Etant donnée la fonction $F^l$ représentant la couche $l$, on définit l'erreur de contenu pour la couche $l$ par l'erreur quadratique moyenne entre les représentations des deux images par cette couche:

$$
\mathcal{L}_\text{content}(x, x_0, l) := \dfrac{1}{2N^l M^l}\left\|F^l(x)-F^l(x_0)\right\|_2^2,
$$

où la dimension spatiale $M^l$ et le nombre de filtres $N^l$ sont les dimensions de la feature map: $F^l(x)\in \mathbb{R}^{N^l\times M^l}.$

La pré-image $x$ est ensuite améliorée par rapport à cette perte par descente de gradient en suivant l'algorithme L-BFGS.

Notons que nous avons omis la pondération $\omega_l$ de cette fonction de perte, qui n'est pas utilisée pour cette tâche par He *et al.*. Nous détaillons et justifions nos autres modifications de cette fonction de perte dans nos commentaires sur ranVGG.

In [ ]:
def content_loss(cnn: nn.Module, layer_outputs: "list[torch.Tensor]", target_outputs: "list[torch.Tensor]", synthetic_image: "torch.Tensor"):
    """Calcule la loss de contenu entre la représentation de l'image 
    synthétique et celle de l'image cible.

    Parameters
    ----------
    cnn : nn.Module
        Modèle à utiliser pour la reconstruction.
    layer_output_ : list[torch.Tensor]
        Liste qui contient les sorties des couches considérées pour la 
        représentation après la forward pass sur cnn 
        (cf. utils.register_model_hooks).
    target_outputs : list[torch.Tensor]
        Représentation de l'image cible.
    synthetic_image : torch.Tensor
        Image synthétique.

    Returns
    -------
    loss: torch.Tensor
    """

    # step 1: forward-propagate and get the layer output from the mutable
    cnn(synthetic_image)

    # step 2: compute the loss
    loss = sum(
        (F.mse_loss(synth_out, target_out)
         for synth_out, target_out in zip(layer_outputs, target_outputs)),
        start=torch.tensor(0)
    )

    return loss


Gatys *et al.* utilisent la première couche convolutive et les 4 premières couches maxpool pour la génération de textures, ce qui motive notre choix de couches utilisées par défaut pour reconstruire l'image :

In [ ]:
# couches utilisées par défaut pour la reconstruction:
TARGET_LAYERS = [1, 4, 9, 18, 27]  # 1rst Conv2d (after ReLU) and MaxPool2d


def reconstruct_image(cnn: nn.Sequential, target_image: torch.Tensor, target_layers: "list[int]" = TARGET_LAYERS, n_steps: int = 5, synth_std: float = 1, logging=False, progressbar=True, synth_init: torch.Tensor = None) -> "tuple[torch.Tensor, torch.Tensor, list[float]]":
    """Reconstruit une image source en inversant sa représentation.
    L'image synthétisée est intialisée par un bruit blanc gaussien.

    Parameters
    ----------
    cnn : nn.Sequential
        Réseau de neurones à utiliser. 
    target_image : torch.Tensor
        Image source à reconstruire.
    target_layers : list[int], default: `TARGET_LAYERS`
        Couches du cnn à utiliser pour reconstruire l'image.
    n_steps : int, default: 5
        Nombre de passes de LBFGS à effectuer. 
    synth_std : float, default: 1
        Ecart-type du bruit blanc gaussien.
    logging : bool, default: False
        Si True, remplit loss_history.
    progressbar : bool, default: True
        Si True, affiche une barre de progression.
    synth_init : torch.Tensor, default: None
        Tenseur à utiliser pour l'intialisation de l'image synthétisée à la 
        place d'un bruit blanc gaussien.

    Returns
    -------
    synthetic_image, final_loss, loss_history
    synthetic_image : torch.Tensor
        Image synthétisée.
    final_loss : torch.Tensor
        Perte de reconstruction associée à l'image synthétisée.
    loss_history : list[float]
        Historique des pertes. Est non vide si et seulement si logging est True.
    """
    if synth_init is not None:
        synthetic_image = synth_init.detach().clone()
        synthetic_image.requires_grad_()
    else:
        # on initialise la pré-image avec un bruit blanc:
        synthetic_image = utils.normal_like(target_image, synth_std)

    layer_outputs, handles = utils.register_model_hooks(
        cnn,
        target_layers
    )

    # le réseau ne va pas être modifié, donc on peut calculer la cible une fois pour toutes:
    cnn(target_image)
    # on considère la cible comme une vérité terrain:
    target_outputs = [t.detach().clone() for t in layer_outputs]

    optimizer = optim.LBFGS(
        [synthetic_image],
        max_iter=20
    )

    loss_history = []
    if progressbar:
        iterator = trange(n_steps, desc="Image reconstruction", unit="step")
    else:
        iterator = range(n_steps)
    for _ in iterator:
        def closure():
            # zero out the gradients, else they'll accumulate in synthetic_image.grad
            optimizer.zero_grad()
            loss = content_loss(
                cnn,
                layer_outputs,
                target_outputs,
                synthetic_image
            )
            loss.backward()  # backpropagate the loss to the input
            if logging:
                loss_history.append(loss.item())
            return loss
        optimizer.step(closure)
        if progressbar and logging:
            iterator.set_postfix(content_loss=loss_history[-1])

    final_loss = content_loss(cnn, layer_outputs, target_outputs, synthetic_image)

    utils.unregister_model_hooks(handles)

    return synthetic_image, final_loss, loss_history


Testons la reconstruction d'image sur le modèle VGG pré-entraîné:

In [ ]:
input_image_name = "briques.png"
img_size = 256

target_image = utils.prep_img(input_image_name, img_size).to(device)
synthetic_image, loss, loss_history = reconstruct_image(
    cnn,
    target_image,
    n_steps=100,
    logging=True
)

fig, _ = plt.subplots(1, 2, figsize=(15, 10))
fig.axes[0].imshow(utils.to_pil(target_image))
fig.axes[0].set_title("Original")
fig.axes[1].imshow(utils.to_pil(synthetic_image))
fig.axes[1].set_title("Reconstruction")
fig.tight_layout()
plt.show()

Nous arrivons bien à reconstruire l'image d'origine.

In [ ]:
# plot loss history
fig, _ = plt.subplots(1, 1, figsize=(15, 10))
fig.axes[0].plot(loss_history)
fig.axes[0].set_xlabel("L-BFGS iterations")
fig.axes[0].set_ylabel("Content loss")
fig.axes[0].set_xscale('log')
plt.show()

Au vu de la courbe ci-dessus, la fonction de perte semble converger pour VGG autour de 100 itérations de L-BGFS; nous avons donc choisi `n_steps = 5` par défaut, ce qui correspond bien à 100 itérations de L-BFGS (`optim.LBFGS` effectue 20 itérations par étape).

### Construction de ranVGG

In [ ]:
# TODO décrire le mode de construction, pseudo-entrainement

In [ ]:
def normalize_layer_(cnn: nn.Sequential, target_image: torch.Tensor, layer_idx: int):
    """Normalise les poids de la couche convolutive de manière à ce que l'activation
    moyenne soit 1.
    """
    activation_idx = layer_idx + 1
    layer_output_, handle_ = utils.register_model_hooks(
        cnn,
        [activation_idx]
    )
    cnn(target_image)
    with torch.no_grad():
        mean_activation = torch.mean(layer_output_[0]).item()
        cnn[layer_idx].weight.copy_(cnn[layer_idx].weight / mean_activation)
        cnn[layer_idx].bias.copy_(cnn[layer_idx].bias / mean_activation)
    utils.unregister_model_hooks(handle_)

In [ ]:
def build_ranvgg_(cnn: nn.Sequential, target_image: torch.Tensor, target_layers: "list[int]" = TARGET_LAYERS, n_samples: int = 5, n_steps: int = 5, synth_std: float = 1, normalize=True):
    # on utilise la même image synthétique pour comparer tous les échantillons:
    synthetic_image = utils.normal_like(target_image, synth_std)
    cnn.requires_grad_(False)  # on empêche le modèle d'apprendre
    # on entraîne les couches convolutives:
    layers_to_build = [
        idx for idx, layer in enumerate(cnn) if isinstance(layer, nn.Conv2d)
    ]
    for layer_idx in tqdm(layers_to_build, desc="Building ranVGG", unit="layer"):
        # on vise la couche d'activation (ReLU) pour la reconstruction d'image:
        activation_idx = layer_idx + 1
        best_layer = None
        best_loss = float("inf")

        with trange(n_samples, leave=False, unit="sample") as pbar:
            for _ in pbar:
                # on tire une nouvelle couche convolutive:
                cnn[layer_idx] = utils.conv2D_like(cnn[layer_idx])
                # on lui donne un écart-type de 0.015:
                utils.randomize_layer_(cnn[layer_idx])
                if normalize:
                    # on normalise les poids de la couche:
                    normalize_layer_(cnn, target_image, layer_idx)

                # on évalue la qualité de notre couche:
                _, loss, _ = reconstruct_image(
                    cnn,
                    target_image,
                    target_layers,  # TODO maybe build a list of current target layers instead ?
                    synth_init=synthetic_image,
                    n_steps=n_steps,
                    progressbar=False
                )
                # on ne garde pas le gradient de la loss:
                loss = loss.item()
                if loss < best_loss:
                    best_loss = loss
                    best_layer = cnn[layer_idx]
                    pbar.set_postfix(best_loss=best_loss)
        cnn[layer_idx] = best_layer


In [ ]:
# construction de ranVGG
input_image_name = "briques.png"
img_size = 256

target_image = utils.prep_img(input_image_name, img_size).to(device)
ranvgg = models.vgg19(pretrained=False).features.to(device).eval()
build_ranvgg_(ranvgg, target_image, n_steps=5, n_samples=5)

In [ ]:
# test de ranVGG

synthetic_image, loss, loss_history = reconstruct_image(
    ranvgg,
    target_image,
    n_steps=20,
    logging=True
)

fig, _ = plt.subplots(1, 2, figsize=(15, 10))
fig.axes[0].imshow(utils.to_pil(target_image))
fig.axes[0].set_title("Original")
fig.axes[1].imshow(utils.to_pil(synthetic_image))
fig.axes[1].set_title("Reconstruction")
fig.tight_layout()
plt.show()

In [ ]:
# plot loss history
fig, _ = plt.subplots(1, 1, figsize=(15, 10))
fig.axes[0].plot(loss_history)
fig.axes[0].set_xlabel("L-BFGS iterations")
fig.axes[0].set_ylabel("Content loss")
fig.axes[0].set_xscale('log')
plt.show()

Dans un premier temps, on remarque que l'image est bien reconstruite par ranVGG. Dans un second temps, on peut constater que contrairement à He *et al.*, nous avons utilisé toutes les couches `TARGET_LAYERS` (rappelées dans le code ci-dessous) pour calculer la fonction de perte. La raison est que malgré la normalisation de chaque couche, on observe le problème de vanishing gradient si on n'utilise que les couches profondes (comme le font He *et al.*) pour calculer la fonction de perte; les réseaux modernes évitent ce problème en utilisant des couches résiduelles (cf. [ResNet](https://arxiv.org/abs/1512.03385)). Pour le gradient, cela revient à sommer les gradients de différentes couches; VGG n'ayant pas de couche résiduelle, notre approche est donc de sommer les pertes de différentes couches pour constituer notre perte de contenu sur le réseau de neurones tout entier:

$$
\mathcal{L}_\text{content}(x_0,x)
:=\sum_{l\in \texttt{TARGET\_LAYERS}} \mathcal{L}_\text{content}(x_0,x, l) 
= \sum_{l\in \texttt{TARGET\_LAYERS}} \dfrac{1}{2N^l M^l}\left\|F^l(x)-F^l(x_0)\right\|_2^2,
$$

On peut tester ce raisonnement en essayant de reconstruire l'image à partir d'une couche à la fois:

In [ ]:
TARGET_LAYERS = [1, 4, 9, 18, 27]  # 1rst Conv2d (after ReLU) and MaxPool2d
LAYER_NAMES = ["conv1", "pool1", "pool2", "pool3", "pool4"]

fig, axes = plt.subplots(len(TARGET_LAYERS) + 1, 2, figsize=(20, 40))
axes[0, 0].imshow(utils.to_pil(target_image))
axes[0, 0].set_title("Original")
for idx, (layer_idx, layer_name) in enumerate(zip(TARGET_LAYERS, LAYER_NAMES)):
    synthetic_image, loss, loss_history = reconstruct_image(
        ranvgg,
        target_image,
        target_layers=[layer_idx],
        n_steps=20,
        logging=True
    )
    axes[idx + 1, 0].imshow(utils.to_pil(synthetic_image))
    axes[idx + 1, 0].set_title(f"Reconstruction ({layer_name})    Loss = {loss.item()}")
    axes[idx + 1, 1].plot(loss_history)
    axes[idx + 1, 1].set_xlabel("L-BFGS iterations")
    axes[idx + 1, 1].set_ylabel("Content loss")
    axes[idx + 1, 1].set_xscale('log')
fig.tight_layout()
plt.show()


On remarque que cette reconstuction a bien convergé dans tous les cas, mais se dégrade au fur et a mesure qu'on augmente en profondeur dans le réseau.

On peut également noter que L-BFGS a une condition d'arrêt qui lui permet d'économiser des itérations si la fonction de perte ou son gradient sont suffisamment faibles.

Une autre expérience intéressante est de regarder l'effet de la normalisation des couches sur le rendu final (cf `normalize_layer_` plus haut); cette normalisation, introduite par Gatys *et al.*, s'assure que:

$$
\dfrac{1}{WHC}\sum_{i=1}^W \sum_{j=1}^H \sum_{k=1}^C F^l(x_0) = 1
$$

Regardons les résultats de reconstruction si l'on omet cette normalisation:

In [ ]:
# construction de ranVGG
input_image_name = "briques.png"
img_size = 256

target_image = utils.prep_img(input_image_name, img_size).to(device)
unnormalized_ranvgg = models.vgg19(pretrained=False).features.to(device).eval()
build_ranvgg_(unnormalized_ranvgg, target_image, n_steps=5, n_samples=5, normalize=False)

# test de ranVGG

synthetic_image, loss, loss_history = reconstruct_image(
    unnormalized_ranvgg,
    target_image,
    n_steps=20,
    logging=True
)

fig, _ = plt.subplots(1, 2, figsize=(15, 10))
fig.axes[0].imshow(utils.to_pil(target_image))
fig.axes[0].set_title("Original")
fig.axes[1].imshow(utils.to_pil(synthetic_image))
fig.axes[1].set_title("Reconstruction")
fig.tight_layout()
plt.show()

### singleVGG

Nous testons ici une approche proposée par Pierrick Chatillon, qui consiste à remplacer le pseudo-entraînement de He *et al.* par une descente de gradient par rapport à la perte de reconstruction, l'idée étant que l'on veut que notre couche puisse plus efficacement reconstruire l'image à partir d'un bruit blanc.

In [ ]:
def build_singlevgg_(cnn: nn.Module, target_image: torch.Tensor, target_layers: "list[int]" = TARGET_LAYERS, n_epochs: int = 20, n_steps: int = 5, synth_std: float = 1):
    cnn.requires_grad_(False)  # on empêche le modèle d'apprendre
    # on entraîne les couches convolutives:
    layers_to_build = [
        idx for idx, layer in enumerate(cnn) if isinstance(layer, nn.Conv2d)
    ]
    for layer_idx in tqdm(layers_to_build, desc="Building ranVGG", unit="layer"):
        conv_layer = cnn[layer_idx]
        normalize_layer_(cnn, target_image, layer_idx)
        # on entraîne chaque layer individuellement:
        conv_layer.requires_grad_(True)
        optimizer = optim.Adam(conv_layer.parameters(), lr=0.0001)
        # on vise la couche d'activation (ReLU) pour la reconstruction d'image:
        activation_idx = layer_idx + 1
        with trange(n_epochs, leave=False, unit="epoch") as pbar:
            for _ in pbar:
                # on évalue la qualité de notre couche:
                # NB: l'image synthétique change à chaque epoch
                _, loss, _ = reconstruct_image(
                    cnn,
                    target_image,
                    target_layers=target_layers,
                    n_steps=n_steps,
                    progressbar=False
                )
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                normalize_layer_(cnn, target_image, layer_idx)
                pbar.set_postfix(loss=loss.item())
        conv_layer.requires_grad_(False)


In [ ]:
# construction de singleVGG
input_image_name = "briques.png"
img_size = 256

target_image = utils.prep_img(input_image_name, img_size).to(device)
singlevgg = models.vgg19(pretrained=False).features.to(device).eval()
build_singlevgg_(singlevgg, target_image, n_steps=5, n_epochs=20)

In [ ]:
# test de singleVGG

synthetic_image, loss, loss_history = reconstruct_image(
    singlevgg,
    target_image,
    n_steps=100,
    logging=True
)

fig, _ = plt.subplots(1, 2, figsize=(15, 10))
fig.axes[0].imshow(utils.to_pil(target_image))
fig.axes[0].set_title("Original")
fig.axes[1].imshow(utils.to_pil(synthetic_image))
fig.axes[1].set_title("Reconstruction")
fig.tight_layout()
plt.show()

In [ ]:
# plot loss history
fig, _ = plt.subplots(1, 1, figsize=(15, 10))
fig.axes[0].plot(loss_history)
fig.axes[0].set_xlabel("L-BFGS iterations")
fig.axes[0].set_ylabel("Content loss")
fig.axes[0].set_xscale('log')
plt.show()

In [ ]:
# TODO comments on performance, training time

In [ ]:
#TODO stuff below here is for later use

In [ ]:
def gramm(tnsr: torch.Tensor) -> torch.Tensor:
    """Computes Gram matrix for the input batch tensor.
    Args: tnsr (torch.Tensor): input tensor of the Size([B, C, H, W]).
    Returns:  G (torch.Tensor): output tensor of the Size([B, C, C]).
    """
    b, c, h, w = tnsr.size()
    F = tnsr.view(b, c, h * w)
    G = torch.bmm(F, F.transpose(1, 2))
    G.div_(h * w)
    return G


def gram_loss(input: torch.Tensor, gramm_target: torch.Tensor, weight: float = 1.0):
    """Computes the MSE loss between the Gram matrix of the input and the target
    Gram matrix. 
    """
    loss = weight * F.mse_loss(gramm(input), gramm_target)
    return loss


#TODO define texture_loss

In [ ]:
# layers to use in the texture synthesis:
TARGET_LAYERS = [1, 4, 9, 18, 27]  # 1rst Conv2d (after ReLU) and MaxPool2d
LAYER_NAMES = ["conv1", "pool1", "pool2", "pool3", "pool4"]